In [9]:
# default_exp importing

# This notebook provides a function to import peptide level data from Spectronaut or MaxQuant

The preprocessed data is stored in a pandas dataframe with following columns:
* all_protein_ids: all UniProt IDs the peptide map to separated by ';'
* modified_sequence: the peptide sequence with all modifications included in square brackets
* naked_sequence: the naked peptide sequence

It is possible to further select one or more specific samples for import. A single sample can be provided as character string. Multiple samples can be provided as list of character strings. The raw MS filename should match corresponding entries in the "R.FileName" or "Raw file" column of the Spectronaut and MaxQuant analysis respectively.

In [10]:
#export
import os
import pandas as pd

def read_file(
    file: str, 
    column_names: list
) -> pd.DataFrame:
    """Load a specified columns of the file as a pandas dataframe.

    Args:
        file (str): The name of a file.
        column_names (list): The list of three columns that should be extracted from the file.

    Returns:
        pd.DataFrame: A pandas dataframe with all the data stored in the specified columns.
    """
    file_ext = os.path.splitext(file)[-1]
    if file_ext=='.csv':
        sep=','
    elif file_ext=='.tsv':
        sep='\t'
    elif file_ext=='.txt':
        sep='\t'

    with open(file) as filelines:
        i = 0
        pos = 0
        for l in filelines:
            i += 1
            l = l.split(sep)
            raw = l.index(column_names[0])
            prot = l.index(column_names[1])
            seq = l.index(column_names[2])
            if i>0:
                break
            
    with open(file) as filelines:
        raws = []
        prots = []
        seqs = []
        for l in filelines:
            l = l.split(sep)
            raws.append(l[raw])
            prots.append(l[prot])
            seqs.append(l[seq])

    res = pd.DataFrame({column_names[0]:raws[1:],
             column_names[1]:prots[1:],
             column_names[2]:seqs[1:]})
    
    return res


def extract_rawfile_unique_values(
    file: str
) -> list:
    """Extract the unique raw file names either from "R.FileName" (Spectronaut output) or "Raw file" (MaxQuant output) column.
    
    Args:
        file (str): The name of a file.

    Returns:
        list: A sorted list of unique raw file names from the file.
    """
    file_ext = os.path.splitext(file)[-1]
    if file_ext == '.csv':
        sep = ',' 
    elif file_ext in ['.tsv', '.txt']:
        sep = '\t'

    with open(file) as filelines:
        i = 0
        filename_col_index = int()
        filename_data = []

        for l in filelines:
            l = l.split(sep)
            # just do it for the first line 
            if i == 0:
                try:
                    filename_col_index = l.index('R.FileName')
                except ValueError as err:
                    filename_col_index = l.index('Raw file')
            else:
                filename_data.append(l[filename_col_index])
            i += 1
            
        unique_filenames = set(filename_data)
        
    sorted_unique_filenames = sorted(list(unique_filenames))
    return sorted_unique_filenames

In [11]:
#hide
def test_extract_rawfile_unique_values():
    data_Spectronaut_csv = "../testdata/test_spectronaut_input.csv"
    data_Spectronaut_tsv = "../testdata/test_spectronaut_input.tsv"
    assert ['raw_01', 'raw_02'] == extract_rawfile_unique_values(data_Spectronaut_csv)
    assert extract_rawfile_unique_values(data_Spectronaut_csv) == extract_rawfile_unique_values(data_Spectronaut_tsv)
    
    data_MQ = "../testdata/test_maxquant_input.txt"
    assert len(extract_rawfile_unique_values(data_MQ)) == 381
    
test_extract_rawfile_unique_values()

## Import Spectronaut data

In [12]:
#export
import pandas as pd
import re
from typing import Union

def import_spectronaut_data(
    file: str, 
    sample: Union[str, list, None] = None
) -> pd.DataFrame:
    """Import peptide level data from Spectronaut. 
    
    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.

    Returns:
        pd.DataFrame: A pandas dataframe containing information about:
            - all_protein_ids (str)
            - modified_sequence (str)
            - naked_sequence (str) 
    """
    spectronaut_columns = ["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence","R.FileName"]
    
    data = read_file(file, spectronaut_columns)
    
    if sample:
        if isinstance(sample, list):
            data_sub = data[data["R.FileName"].isin(sample)]
            data_sub = data_sub[["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence"]]
        elif isinstance(sample, str):
            data_sub = data[data["R.FileName"] == sample]
            data_sub = data_sub[["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence"]]
    else:
        data_sub = data[["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence"]]

    # get modified sequence
    mod_seq = data_sub.apply(lambda row: re.sub('_','',row["EG.ModifiedSequence"]), axis=1)
    data_sub = data_sub.assign(modified_sequence=mod_seq.values)
    # get naked sequence
    nak_seq = data_sub.apply(lambda row: re.sub(r'\[.*?\]','',row["modified_sequence"]), axis=1)
    data_sub = data_sub.assign(naked_sequence=nak_seq.values)
    data_sub = data_sub.rename(columns={"PEP.AllOccurringProteinAccessions": "all_protein_ids"})
    input_data = data_sub[["all_protein_ids","modified_sequence","naked_sequence"]]
    input_data = input_data.dropna()
    input_data = input_data.drop_duplicates().reset_index(drop=True)
    return input_data

In [13]:
#hide

def test_import_spectronaut_data():
    # test entire input test data
    data = import_spectronaut_data("../testdata/test_spectronaut_input.csv")
    #print(data.shape[0])
    assert data.shape[0] == 40
    data_t = import_spectronaut_data("../testdata/test_spectronaut_input.tsv")
    #print(data_t.shape[0])
    pd.testing.assert_frame_equal(data, data_t)
    test = pd.read_csv('../testdata/test_spectronaut_imported.csv', sep=',') 
    #print(test.shape[0])
    pd.testing.assert_frame_equal(data, test)
    
    # test single sample
    data = import_spectronaut_data("../testdata/test_spectronaut_input.csv", 
                                   sample="raw_01")
    #print(data.shape[0])
    assert data.shape[0] == 40
    data = import_spectronaut_data("../testdata/test_spectronaut_input.csv", 
                                   sample="raw_02")
    #print(data.shape[0])
    assert data.shape[0] == 20
    
    # test multiple samples
    data = import_spectronaut_data("../testdata/test_spectronaut_input.csv", 
                                   sample=["raw_01","raw_02"])
    #print(data.shape[0])
    assert data.shape[0] == 40

test_import_spectronaut_data()

## Import MaxQuant data

In [14]:
#export
import pandas as pd
import re

def import_maxquant_data(
    file: str, 
    sample: Union[str, list, None] = None
) -> pd.DataFrame:
    """Import peptide level data from MaxQuant.
    
    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.

    Returns:
        pd.DataFrame: A pandas dataframe containing information about:
            - all_protein_ids (str)
            - modified_sequence (str)
            - naked_sequence (str) 
    """
    mq_columns = ["Proteins","Modified sequence","Raw file"]

    data = read_file(file, mq_columns)
    
    if sample:
        if isinstance(sample, list):
            data_sub = data[data["Raw file"].isin(sample)]
            data_sub = data_sub[["Proteins","Modified sequence"]]
        elif isinstance(sample, str):
            data_sub = data[data["Raw file"] == sample]
            data_sub = data_sub[["Proteins","Modified sequence"]]
    else:
        data_sub = data[["Proteins","Modified sequence"]]
        
    # get modified sequence
    mod_seq = data_sub.apply(lambda row: re.sub('_','',row["Modified sequence"]), axis=1)
    data_sub = data_sub.assign(modified_sequence=mod_seq.values)
    
    # replace outer () with []
    mod_seq_replaced = data_sub.apply(lambda row: re.sub(r'\((.*?\(.*?\))\)',r'[\1]',row["modified_sequence"]), axis=1)
    data_sub = data_sub.assign(modified_sequence=mod_seq_replaced.values)
    
    # get naked sequence
    nak_seq = data_sub.apply(lambda row: re.sub(r'\[.*?\]','',row["modified_sequence"]), axis=1)
    data_sub = data_sub.assign(naked_sequence=nak_seq.values)
    data_sub = data_sub.rename(columns={"Proteins": "all_protein_ids"})
    input_data = data_sub[["all_protein_ids","modified_sequence","naked_sequence"]]
    input_data = input_data.dropna() # remove missing values
    input_data = input_data.drop_duplicates().reset_index(drop=True)
    return input_data

In [15]:
#hide

def test_import_maxquant_data():
    data = import_maxquant_data("../testdata/test_maxquant_input.txt")
    test = pd.read_csv('../testdata/test_maxquant_imported.csv', sep=',') 
    pd.testing.assert_frame_equal(data, test)
    
    data_s = import_maxquant_data("../testdata/test_maxquant_input.txt", 
                                  sample = "raw_1")
    assert data_s.shape[0] == 85
    
    data_s = import_maxquant_data("../testdata/test_maxquant_input.txt", 
                                  sample = "raw_2")
    assert data_s.shape[0] == 77
    
    data_s = import_maxquant_data("../testdata/test_maxquant_input.txt", 
                                  sample = ["raw_1", "raw_2"])
    assert data_s.shape[0] == 136

test_import_maxquant_data()

## Import AlphaPept data

In [16]:
#export
import re

def convert_ap_mq_mod(
    sequence:str
) -> str:
    """Convert AlphaPept style modifications into MaxQuant style modifications.
    
    Args:
        sequence (str): The peptide sequence with modification in an AlphaPept style.

    Returns:
        str: The peptide sequence with modification in a similar to MaxQuant style.
    """
    modif_convers_dict = {
        '[ox]': '[Oxidation (M)]', '[a]': '[Acetylation]', '[am]': '[Amidation]', '[p]': '[Phosphorylation]',
        '[pg]': '[Pyrrolidone carboxylic acid]', '[c]': '[Disulfide bond]'
    }
    mods = re.findall('[a-z0-9]+', sequence)
    if mods:
        for mod in mods:
            i = sequence.index(mod)
            sequence = sequence.replace(mod, '')
            sequence = sequence[:i+1] + f'[{mod}]' + sequence[i+1:]
        for k,v in modif_convers_dict.items():
            if k in sequence:
                sequence = sequence.replace(k, v)
    return sequence

In [17]:
#hide

def test_convert_ap_mq_mod():
    seq1 = 'cCVNTTLQIK'
    assert "C[Disulfide bond]VNTTLQIK" == convert_ap_mq_mod(seq1)
    seq2 = 'HAEoxMVHTGLK' 
    assert "HAEM[Oxidation (M)]VHTGLK" == convert_ap_mq_mod(seq2)
    seq3_several_mods = 'AcCLDYPVTSVLPPASLoxMK'
    assert "AC[Disulfide bond]LDYPVTSVLPPASLM[Oxidation (M)]K" == convert_ap_mq_mod(seq3_several_mods)
    seq4_no_mod = 'CVNTTLQIK'
    assert "CVNTTLQIK" == convert_ap_mq_mod(seq4_no_mod)
    
test_convert_ap_mq_mod()

In [18]:
#export
import pandas as pd

def import_alphapept_data(
    file: str, 
    sample: Union[str, list, None] = None
) -> pd.DataFrame:
    """Import peptide level data from AlphaPept.
    
    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.

    Returns:
        pd.DataFrame: A pandas dataframe containing information about:
            - all_protein_ids (str)
            - modified_sequence (str)
            - naked_sequence (str) 
    """
    ap_columns = ["protein_group", "sequence", "shortname"]
    
    data = read_file(file, ap_columns)
    
    if sample:
        if isinstance(sample, list):
            data_sub = data[data["shortname"].isin(sample)]
            data_sub = data_sub[["protein_group", "sequence"]]
        elif isinstance(sample, str):
            data_sub = data[data["shortname"] == sample]
            data_sub = data_sub[["protein_group", "sequence"]]
    else:
        data_sub = data[["protein_group", "sequence"]]
    
    data_sub = data_sub[~data_sub.sequence.str.contains('_decoy')]
    
    # get modified sequence
    modif_seq = data_sub.apply(lambda row: convert_ap_mq_mod(row.sequence), axis=1)
    data_sub['modified_sequence'] = modif_seq.values
    
    # get a list of proteins_id
    proteins = data_sub.apply(lambda row: ",".join([_.split('|')[1] for _ in row.protein_group.split(',')]), axis=1)
    data_sub['all_protein_ids'] = proteins.values
    
    # get naked sequence
    nak_seq = data_sub.apply(lambda row: ''.join([_ for _ in row.sequence if _.isupper()]), axis=1)
    data_sub['naked_sequence'] = nak_seq.values

    input_data = data_sub[["all_protein_ids", "modified_sequence", "naked_sequence"]]
    input_data = input_data.dropna() # remove missing values
    input_data = input_data.drop_duplicates().reset_index(drop=True)
    return input_data

## Aggregated import function

In [19]:
#export
import pandas as pd
import re
from io import StringIO
import os

def import_data(
    file :str, 
    sample: Union[str, list, None] = None,
    verbose: bool = True, 
    dashboard: bool = False
) -> pd.DataFrame:
    """Import peptide level data. Depending on available columns in the provided file, 
    the function calls other specific functions for each tool. 
    
    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.
        verbose (bool): if True, print the type of input that is used. Defaults to True.
        dashboard (bool): If True, the function is used for the dashboard output (StringIO object). Defaults to False.
     
    Raises:
        TypeError: If the input data format is unknown.
    
    Returns:
        pd.DataFrame: A pandas dataframe containing information about:
            - all_protein_ids (str)
            - modified_sequence (str)
            - naked_sequence (str) 
    """
    if dashboard:
        file = StringIO(str(file, "utf-8"))
    
    file_ext = os.path.splitext(file)[-1]
    if file_ext=='.csv':
        sep=','
    elif file_ext=='.tsv':
        sep='\t'
    elif file_ext=='.txt':
        sep='\t'

    with open(file) as filelines:
        i = 0
        pos = 0
        for l in filelines:
            i += 1
            l = l.split(sep)
            if i>0:
                break
        
        uploaded_data_columns = set(l)
        input_info = file
    if set(["Proteins","Modified sequence","Raw file"]).issubset(uploaded_data_columns):
        if verbose:
            print("Import MaxQuant input")
        data = import_maxquant_data(input_info, sample=sample)
    elif set(["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence","R.FileName"]).issubset(uploaded_data_columns):
        if verbose:
            print("Import Spectronaut input")
        data = import_spectronaut_data(input_info, sample=sample)
    elif set(["protein_group", "sequence", "shortname"]).issubset(uploaded_data_columns):
        if verbose:
            print("Import AlphaPept input")
        data = import_alphapept_data(input_info, sample=sample)
    else: 
        raise TypeError(f'Input data format for {file} not known.')
    return data

In [20]:
#hide
import sys

def test_import_data():
    data_MQ = import_data("../testdata/test_maxquant_input.txt", verbose=False)
    test = pd.read_csv('../testdata/test_maxquant_imported.csv', sep=',') 
    pd.testing.assert_frame_equal(data_MQ, test)
    
    data_S_csv = import_data("../testdata/test_spectronaut_input.csv", verbose=False)
    data_S_tsv = import_data("../testdata/test_spectronaut_input.tsv", verbose=False)
    pd.testing.assert_frame_equal(data_S_csv, data_S_tsv)
    test = pd.read_csv('../testdata/test_spectronaut_imported.csv', sep=',') 
    pd.testing.assert_frame_equal(data_S_csv, test)
    
    data_S_sub = import_data("../testdata/test_spectronaut_input.csv", 
                             sample = "raw_01", 
                             verbose=False)
    assert data_S_sub.shape[0] == 40
    
    try:
        out = import_data("../testdata/test_uniprot_df.csv")
    except TypeError as e:
        out = e
    assert str(out) == "Input data format for ../testdata/test_uniprot_df.csv not known."    

test_import_data()

## Export notebook to script

In [21]:
#hide
from nbdev.showdoc import *

In [22]:
#hide
from nbdev.export import *
notebook2script()

Converted Importing.ipynb.
Converted index.ipynb.
Converted organisms_data.ipynb.
Converted Preprocessing.ipynb.
Converted proteolytic_cleavage.ipynb.
Converted SequencePlot.ipynb.
Converted Uniprot_integration.ipynb.
